In [2]:
!pip install transformers datasets peft torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [3]:
from datasets import load_dataset

# Load the IMDB dataset
dataset = load_dataset("imdb")

# Take 10,000 reviews for training
train_dataset = dataset["train"].shuffle(seed=42).select(range(10000))

# Take 2,000 reviews for testing
test_dataset = dataset["test"].shuffle(seed=42).select(range(2000))

# Look at the data
print("Training dataset size:", len(train_dataset))
print("Test dataset size:", len(test_dataset))
print("Example training review:", train_dataset[0])
print("Example test review:", test_dataset[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Training dataset size: 10000
Test dataset size: 2000
Example training review: {'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}
Example test review: {'text': "<br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear

In [4]:
from transformers import AutoTokenizer

# Load the tokenizer for DistilBERT
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to tokenize text
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize both datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Check one tokenized example
print(tokenized_train[0])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1, 'input_ids': [101, 2045, 2003, 2053, 7189, 2012, 2035, 2090, 3481, 3771, 1998, 6337, 2099, 2021, 1996, 2755, 2008, 2119, 2024, 2610, 2186, 2055, 6355, 6997, 1012, 6337, 2099, 3504, 15594, 2100, 1010, 3481, 3771, 350

In [5]:
from transformers import AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

# Load the DistilBERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Set up LoRA
lora_config = LoraConfig(
    r=8,  # How much to adjust the model (small number = less memory)
    lora_alpha=16,  # A tuning knob for LoRA
    target_modules=["q_lin", "v_lin"],  # Which parts of the model to tweak
    lora_dropout=0.1,  # Helps prevent overfitting
    bias="none"  # Simplifies the model
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Show how many parameters we’re training
model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 147,456 || all params: 67,102,466 || trainable%: 0.2197


In [6]:
from transformers import TrainingArguments, Trainer
import numpy as np

# Function to compute accuracy with label names
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

# Set up training settings
training_args = TrainingArguments(
    output_dir="./results",  # Save results
    eval_strategy="epoch",  # Check after each epoch
    learning_rate=2e-5,  # Small learning steps
    per_device_train_batch_size=16,  # Process 16 reviews
    per_device_eval_batch_size=16,  # Same for testing
    num_train_epochs=3,  # 3 passes through data
    weight_decay=0.01,  # Prevents overfitting
    logging_dir="./logs",  # Save logs
    logging_steps=100,  # Show progress every 100 steps
    run_name="sentiment_analysis_run",  # Unique name for W&B
    report_to="none"  # Disable W&B
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics  # Add label names and accuracy
)

# Start training
trainer.train()

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,0.604300,No log
2,0.437000,No log
3,0.403000,No log


TrainOutput(global_step=1875, training_loss=0.514553834025065, metrics={'train_runtime': 1164.8126, 'train_samples_per_second': 25.755, 'train_steps_per_second': 1.61, 'total_flos': 3987611504640000.0, 'train_loss': 0.514553834025065, 'epoch': 3.0})

In [7]:
# Check accuracy
results = trainer.evaluate()
print("Test results:", results)

Test results: {'eval_runtime': 30.7202, 'eval_samples_per_second': 65.104, 'eval_steps_per_second': 4.069, 'epoch': 3.0}


In [12]:
import torch

# Function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    device = torch.device("cuda" if torch.cuda.is_available() and next(model.parameters()).is_cuda else "cpu")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    return "Positive" if prediction == 1 else "Negative"

# Interactive input
while True:
    user_input = input("Enter a sentence to analyze (or type 'quit' to stop): ")
    if user_input.lower() == 'quit':
        break
    print("Sentiment:", predict_sentiment(user_input))

Enter a sentence to analyze (or type 'quit' to stop): I love this new phone!
Sentiment: Positive
Enter a sentence to analyze (or type 'quit' to stop): This service is terrible.
Sentiment: Negative
Enter a sentence to analyze (or type 'quit' to stop): The movie was okay, I guess.
Sentiment: Negative
Enter a sentence to analyze (or type 'quit' to stop): wow
Sentiment: Positive
Enter a sentence to analyze (or type 'quit' to stop): amazing
Sentiment: Positive
Enter a sentence to analyze (or type 'quit' to stop): eww
Sentiment: Negative
Enter a sentence to analyze (or type 'quit' to stop): you are improving
Sentiment: Positive
Enter a sentence to analyze (or type 'quit' to stop): i cant see you
Sentiment: Negative
Enter a sentence to analyze (or type 'quit' to stop): quit


In [13]:
# Save model and tokenizer
model.save_pretrained("./my_sentiment_model")
tokenizer.save_pretrained("./my_sentiment_model")

('./my_sentiment_model/tokenizer_config.json',
 './my_sentiment_model/special_tokens_map.json',
 './my_sentiment_model/vocab.txt',
 './my_sentiment_model/added_tokens.json',
 './my_sentiment_model/tokenizer.json')

In [14]:
from google.colab import files
import shutil

# Zip and download
shutil.make_archive("my_sentiment_model", "zip", "./my_sentiment_model")
files.download("my_sentiment_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>